# Lesson: Building Shell-Enabled Agents with `ShellToolMiddleware`

This notebook shows how to configure shell access for LangChain agents in three practical ways: basic host execution, a restricted/secure setup, and a maintenance-focused shell profile.

> **Teaching goal:** understand not just *what* to type, but *why* each middleware option exists and when to use it.


## Learning Goals

By the end of this lesson, you should be able to:

- Explain the role of `ShellToolMiddleware` in an agent workflow.
- Configure a basic shell-enabled agent on the host machine.
- Add security controls like environment variables, startup commands, and redaction.
- Customize shell behavior (shell type, tool description, shutdown cleanup).
- Trace the end-to-end script flow from setup to invocation and output.

## Prerequisite Knowledge

Before running this notebook, you should be comfortable with:

- Basic Python syntax (imports, variables, dictionaries, function calls)
- The idea of an LLM-powered agent
- Running local models with Ollama
- Basic shell commands (`echo`, `touch`, file listing)


## Script Walkthrough (Big Picture)

The script follows this flow:

1. **Setup** imports, environment variables, and the local LLM.
2. **Example 1** creates a basic shell-enabled agent and runs a simple file task.
3. **Example 2** creates a restricted workspace with env vars + redaction rules.
4. **Example 3** creates a maintenance-oriented shell profile with cleanup commands.
5. Each example calls `agent.invoke(...)` and prints the final response.

**Why this matters:** this pattern is reusable for real systems where agents need controlled shell automation.


## Section 1 — Environment and Model Setup

This first block imports dependencies, loads `.env` values, and initializes `ChatOllama`.

### Inline Notes

- `load_dotenv()` loads environment variables from a local `.env` file.
- `os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")` provides a safe default when the variable is missing.
- `model="gpt-oss:20b"` selects the local model used by all examples.

**Design decision:** the LLM is created once and reused, which keeps the notebook cleaner and avoids repeated setup.


In [2]:
# ----------------- MASTERING SHELL TOOL MIDDLEWARE - TUTORIAL CODE -----------------
# This script demonstrates 3 key configurations for ShellToolMiddleware:
# 1. Basic Host Access
# 2. Secure/Restricted Environment (Redaction & Env Vars)
# 3. Custom Maintenance Shell (Zsh & Cleanup)

import os
from dotenv import load_dotenv
from rich import print
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain.agents.middleware import (
    ShellToolMiddleware,
    HostExecutionPolicy,
    RedactionRule,
)

# 1. Setup Environment
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
load_dotenv()

# Initialize Local LLM
llm = ChatOllama(
    model="gpt-oss:20b",
    base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
)

### Common Mistakes to Avoid (Setup)

- Forgetting to run Ollama locally before invoking the agent.
- Misspelling environment variable names (for example `OLLAMA_BASE_URL`).
- Assuming shell middleware works without explicitly adding `ShellToolMiddleware`.

### Quick Checkpoint

1. What is the benefit of giving `os.getenv` a default value?
2. Why is it useful to initialize `llm` once at the top?


## Section 2 — Example 1: Basic Host Shell Agent

This block creates the simplest shell-enabled agent.

### What this code does

- Builds an agent with `create_agent(...)`.
- Attaches `ShellToolMiddleware`.
- Sets `workspace_root="./"` so shell commands start in the current project folder.
- Uses `HostExecutionPolicy()` so commands execute directly on your machine.

**Why this matters:** this is the minimum viable setup for shell automation.


In [3]:
# ---------------- EXAMPLE 1: BASIC HOST SHELL AGENT -----------------
# The simplest configuration. Grants the agent access to the host system to run commands.

print("--- 1. Running Basic Host Agent ---")

basic_agent = create_agent(
    model=llm,
    tools=[],  # We can add other tools here if needed
    middleware=[
        ShellToolMiddleware(
            # workspace_root: Directory where the shell session starts
            workspace_root="./",
            # execution_policy: Controls HOW commands are run (Host vs Docker)
            execution_policy=HostExecutionPolicy(),
        ),
    ],
)

--- 1. Running Basic Host Agent ---

### Section 2B — Running Example 1

The next block sends a human instruction to the agent:

- Create `hello.txt` with content.
- List files.
- Print full result and final assistant message.

### Inline Notes

- `agent.invoke({"messages": [HumanMessage(...)]})` is the standard message-based call format.
- `result["messages"][-1].content` retrieves the last model response for easier reading.


In [4]:
# The agent will create a file and list the directory contents
result_1 = basic_agent.invoke(
    {
        "messages": [
            HumanMessage(
                "Create a file 'hello.txt' with text 'Hello World', then list files."
            )
        ]
    }
)

print(result_1)
print(result_1["messages"][-1].content)

{
    'messages': [
        HumanMessage(
            content="Create a file 'hello.txt' with text 'Hello World', then list files.",
            additional_kwargs={},
            response_metadata={},
            id='7c8bf2ef-5659-4958-8b07-f7f1be76485a'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'gpt-oss:20b',
                'created_at': '2026-02-18T07:00:11.460479236Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 432091660,
                'load_duration': 113322380,
                'prompt_eval_count': 251,
                'prompt_eval_duration': 30882165,
                'eval_count': 74,
                'eval_duration': 267802745,
                'model_name': 'gpt-oss:20b'
            },
            id='lc_run--019c6f8c-982d-7c12-8061-46656308f628-0',
            tool_calls=[
                {
                    'name': 'shell',
                    'args': {'command': 'echo "Hello World" > hello.txt && ls -l', 'restart': False},
                    'id': 'af1f32c3-a3b3-4f23-bf4f-474485885f86',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 251, 'output_tokens': 74, 'total_tokens': 325}
        ),
        ToolMessage(
            content='total 240\n-rw-r--r--@ 1 james  staff  107175 Feb 18 02:00 17-shell_tool.ipynb\n-rw-r--r--@ 1 
james  staff    4466 Feb 18 01:59 17-shell_tool.py\ndrwxr-xr-x@ 3 james  staff      96 Feb 17 15:49 
__pycache__\n-rw-------@ 1 james  staff      12 Feb 18 02:00 hello.txt\ndrwxr-xr-x@ 4 james  staff     128 Feb 17 
16:09 restricted_data\n',
            name='shell',
            id='4701e5c6-867f-4065-8e46-64f5dd9f08a6',
            tool_call_id='af1f32c3-a3b3-4f23-bf4f-474485885f86',
            artifact={
                'timed_out': False,
                'exit_code': 0,
                'truncated_by_lines': False,
                'truncated_by_bytes': False,
                'total_lines': 6,
                'total_bytes': 325,
                'redaction_matches': {}
            }
        ),
        AIMessage(
            content='✅ **hello.txt** has been created with the content:\n\n```\nHello World\n```\n\nHere is the 
current file listing:\n\n```\ntotal 240\n-rw-r--r--@ 1 james  staff  107175 Feb 18 02:00 
17-shell_tool.ipynb\n-rw-r--r--@ 1 james  staff    4466 Feb 18 01:59 17-shell_tool.py\ndrwxr-xr-x@ 3 james  staff  
96 Feb 17 15:49 __pycache__\n-rw-------@ 1 james  staff      12 Feb 18 02:00 hello.txt\ndrwxr-xr-x@ 4 james  staff 
128 Feb 17 16:09 restricted_data\n```',
            additional_kwargs={},
            response_metadata={
                'model': 'gpt-oss:20b',
                'created_at': '2026-02-18T07:00:12.55799756Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1015461926,
                'load_duration': 121579774,
                'prompt_eval_count': 439,
                'prompt_eval_duration': 24259499,
                'eval_count': 221,
                'eval_duration': 809545077,
                'model_name': 'gpt-oss:20b'
            },
            id='lc_run--019c6f8c-9a88-7203-9dda-830f904ee2fb-0',
            tool_calls=[],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 439, 'output_tokens': 221, 'total_tokens': 660}
        )
    ]
}

✅ **hello.txt** has been created with the content:

```
Hello World
```

Here is the current file listing:

```
total 240
-rw-r--r--@ 1 james  staff  107175 Feb 18 02:00 17-shell_tool.ipynb
-rw-r--r--@ 1 james  staff    4466 Feb 18 01:59 17-shell_tool.py
drwxr-xr-x@ 3 james  staff      96 Feb 17 15:49 __pycache__
-rw-------@ 1 james  staff      12 Feb 18 02:00 hello.txt
drwxr-xr-x@ 4 james  staff     128 Feb 17 16:09 restricted_data
```

### Quick Checkpoint (Example 1)

- Which line grants shell capability to the agent?
- What risk comes with direct host execution in production?


## Section 3 — Example 2: Secure / Restricted Agent

This section introduces safer defaults and output protection.

### What this code does

- Creates a dedicated restricted folder.
- Starts shell sessions inside that folder.
- Injects environment variables (including an API key example).
- Runs startup commands automatically.
- Applies redaction rules to hide sensitive output patterns.

### Syntax + Design Notes

- `env={...}` passes values into the shell process environment.
- `startup_commands=[...]` executes initialization commands once per session.
- `RedactionRule(..., detector=r"...")` uses regex to mask matching sensitive text.

**Why this matters:** secure defaults reduce accidental leakage when agents execute shell commands.


In [5]:
# ---------------- EXAMPLE 2: SECURE / RESTRICTED AGENT -----------------
# Demonstrates how to:
# - Set environment variables
# - Run startup commands
# - Redact sensitive information (PII)

print("\n--- 2. Running Secure Agent ---")

# Create a restricted directory for this example
restricted_dir = "./restricted_data"
if not os.path.exists(restricted_dir):
    os.makedirs(restricted_dir)

# In a secure agent, we can set environment variables, run startup commands, and define redaction rules to protect sensitive information.
secure_agent = create_agent(
    model=llm,
    tools=[],
    middleware=[
        ShellToolMiddleware(
            workspace_root=restricted_dir,
            # env: Inject secrets or config into the shell session
            env={"MODE": "restricted", "API_KEY": "sk-secret-12345"},
            # startup_commands: Run these immediately when the session starts
            startup_commands=["echo 'Secure Session Started'", "touch session.lock"],
            # redaction_rules: Regex patterns to hide sensitive output
            redaction_rules=[
                RedactionRule(pii_type="custom_api_key", detector=r"API_KEY=[\w-]+")
            ],
            execution_policy=HostExecutionPolicy(),
        ),
    ],
)

--- 2. Running Secure Agent ---

### Section 3B — Running Example 2

This block asks the agent to inspect an env var and list files in the restricted workspace, then prints the response.

### Common Mistakes to Avoid (Security)

- Writing broad regex redaction patterns that hide too much useful output.
- Storing real secrets directly in notebooks.
- Forgetting that startup commands can fail if paths or permissions are wrong.


In [6]:
result_2 = secure_agent.invoke(
    {"messages": [HumanMessage("Check the API_KEY env var and list files.")]}
)
print(result_2)
print(result_2["messages"][-1].content)

{
    'messages': [
        HumanMessage(
            content='Check the API_KEY env var and list files.',
            additional_kwargs={},
            response_metadata={},
            id='c3d9b6d3-c6cf-4191-8426-57391736e04d'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'gpt-oss:20b',
                'created_at': '2026-02-18T07:00:15.101126426Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 426189282,
                'load_duration': 128234058,
                'prompt_eval_count': 244,
                'prompt_eval_duration': 12198524,
                'eval_count': 72,
                'eval_duration': 260846178,
                'model_name': 'gpt-oss:20b'
            },
            id='lc_run--019c6f8c-a6be-7f32-aff3-5c74ebb47bcf-0',
            tool_calls=[
                {
                    'name': 'shell',
                    'args': {'command': 'echo $API_KEY && ls -la', 'restart': False},
                    'id': 'f3720aa4-4578-43af-aedb-61d96160b142',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 244, 'output_tokens': 72, 'total_tokens': 316}
        ),
        ToolMessage(
            content='sk-secret-12345\ntotal 0\ndrwxr-xr-x@ 4 james  staff  128 Feb 17 16:09 .\ndrwx------@ 8 james 
staff  256 Feb 18 01:59 ..\n-rw-r--r--@ 1 james  staff    0 Feb 18 02:00 session.lock\n-rw-r--r--@ 1 james  staff  
0 Feb 17 16:06 session.log\n',
            name='shell',
            id='b0739e24-b096-4352-aaed-2fa4d94a6815',
            tool_call_id='f3720aa4-4578-43af-aedb-61d96160b142',
            artifact={
                'timed_out': False,
                'exit_code': 0,
                'truncated_by_lines': False,
                'truncated_by_bytes': False,
                'total_lines': 6,
                'total_bytes': 234,
                'redaction_matches': {}
            }
        ),
        AIMessage(
            content='The `API_KEY` environment variable is set to:\n\n```\nsk-secret-12345\n```\n\nThe current 
working directory contains only the session control files:\n\n```\ntotal 0\ndrwxr-xr-x@ 4 james  staff  128 Feb 17 
16:09 .\ndrwx------@ 8 james  staff  256 Feb 18 01:59 ..\n-rw-r--r--@ 1 james  staff    0 Feb 18 02:00 
session.lock\n-rw-r--r--@ 1 james  staff    0 Feb 17 16:06 session.log\n```\n\nNo other files or directories are 
present. If you need to inspect other directories or create files, let me know!',
            additional_kwargs={},
            response_metadata={
                'model': 'gpt-oss:20b',
                'created_at': '2026-02-18T07:00:16.180229111Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1004463637,
                'load_duration': 123619077,
                'prompt_eval_count': 391,
                'prompt_eval_duration': 21498105,
                'eval_count': 217,
                'eval_duration': 794319795,
                'model_name': 'gpt-oss:20b'
            },
            id='lc_run--019c6f8c-a8ba-7413-915a-3130a5af33a1-0',
            tool_calls=[],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 391, 'output_tokens': 217, 'total_tokens': 608}
        )
    ]
}

The `API_KEY` environment variable is set to:

```
sk-secret-12345
```

The current working directory contains only the session control files:

```
total 0
drwxr-xr-x@ 4 james  staff  128 Feb 17 16:09 .
drwx------@ 8 james  staff  256 Feb 18 01:59 ..
-rw-r--r--@ 1 james  staff    0 Feb 18 02:00 session.lock
-rw-r--r--@ 1 james  staff    0 Feb 17 16:06 session.log
```

No other files or directories are present. If you need to inspect other directories or create files, let me know!

### Quick Checkpoint (Example 2)

1. Why isolate shell work inside `./restricted_data`?
2. What is the difference between `env` and `startup_commands`?


## Section 4 — Example 3: Custom Maintenance Shell

This section configures a shell profile tailored for maintenance workflows.

### What this code does

- Provides a custom `tool_description` to guide the model.
- Chooses `shell_command="/bin/zsh"` explicitly.
- Defines `shutdown_commands` for automatic cleanup at session end.

**Why this matters:** explicit shell choice and cleanup behavior make automation more predictable.


In [7]:
# EXAMPLE 3: CUSTOM MAINTENANCE SHELL
#####################################################################
# Demonstrates how to:
# - Use a specific shell (zsh vs bash)
# - Provide a custom tool description for the LLM
# - Run shutdown commands for cleanup

print("\n--- 3. Running Maintenance Agent ---")
# This agent is designed for system maintenance tasks. It uses zsh and has specific cleanup commands that run when the session ends.
maintenance_agent = create_agent(
    model=llm,
    tools=[],
    middleware=[
        ShellToolMiddleware(
            # tool_description: Helps the LLM understand when/how to use this shell
            tool_description="System maintenance shell for cleanup and updates.",
            # shell_command: Explicitly use zsh (or any other shell executable)
            shell_command="/bin/zsh",
            # shutdown_commands: Run these when the agent session closes
            shutdown_commands=[
                "rm -rf ./temp_logs",
                "echo 'Cleanup Complete'",
            ],
            execution_policy=HostExecutionPolicy(),
        ),
    ],
)

--- 3. Running Maintenance Agent ---

### Section 4B — Running Example 3

This block asks for shell version and disk usage, then prints structured and final outputs.

### Inline Notes

- Shutdown commands are useful for removing temporary artifacts.
- Tool descriptions can improve how the LLM decides when to use shell operations.


In [8]:
result_3 = maintenance_agent.invoke(
    {"messages": [HumanMessage("Check shell version and disk usage.")]}
)
print(result_3)
print(result_3["messages"][-1].content)

{
    'messages': [
        HumanMessage(
            content='Check shell version and disk usage.',
            additional_kwargs={},
            response_metadata={},
            id='7cb0e528-f1ac-4b42-a1e1-5cc85667daa8'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'gpt-oss:20b',
                'created_at': '2026-02-18T07:00:19.535577358Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1239537663,
                'load_duration': 129488053,
                'prompt_eval_count': 133,
                'prompt_eval_duration': 15818944,
                'eval_count': 279,
                'eval_duration': 1022038650,
                'model_name': 'gpt-oss:20b'
            },
            id='lc_run--019c6f8c-b4b0-7591-a1a7-fb59b5734ec4-0',
            tool_calls=[
                {
                    'name': 'shell',
                    'args': {'command': 'echo $SHELL; bash --version; df -h', 'restart': False},
                    'id': '1e3f45d8-4f32-482e-ac17-eafd6a776d26',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 133, 'output_tokens': 279, 'total_tokens': 412}
        ),
        ToolMessage(
            content='\nGNU bash, version 3.2.57(1)-release (arm64-apple-darwin25)\nCopyright (C) 2007 Free Software
Foundation, Inc.\nFilesystem        Size    Used   Avail Capacity iused ifree %iused  Mounted on\n/dev/disk3s1s1   
1.8Ti    11Gi   561Gi     2%    453k  4.3G    0%   /\ndevfs            213Ki   213Ki     0Bi   100%     738     0  
100%   /dev\n/dev/disk3s6     1.8Ti    20Ki   561Gi     1%       0  5.9G    0%   /System/Volumes/VM\n/dev/disk3s2  
1.8Ti   7.7Gi   561Gi     2%    1.4k  5.9G    0%   /System/Volumes/Preboot\n/dev/disk3s4     1.8Ti    52Mi   561Gi 
1%     107  5.9G    0%   /System/Volumes/Update\n/dev/disk2s2     500Mi   6.0Mi   483Mi     2%       1  4.9M    0% 
/System/Volumes/xarts\n/dev/disk2s1     500Mi   5.6Mi   483Mi     2%      32  4.9M    0%   
/System/Volumes/iSCPreboot\n/dev/disk2s3     500Mi   544Ki   483Mi     1%      62  4.9M    0%   
/System/Volumes/Hardware\n/dev/disk3s5     1.8Ti   1.2Ti   561Gi    70%     10M  5.9G    0%   
/System/Volumes/Data\nmap auto_home      0Bi     0Bi     0Bi   100%       0     0     -   
/System/Volumes/Data/home\n',
            name='shell',
            id='5554956c-251f-4979-96db-d8eb2110a6e6',
            tool_call_id='1e3f45d8-4f32-482e-ac17-eafd6a776d26',
            artifact={
                'timed_out': False,
                'exit_code': 0,
                'truncated_by_lines': False,
                'truncated_by_bytes': False,
                'total_lines': 14,
                'total_bytes': 1063,
                'redaction_matches': {}
            }
        ),
        AIMessage(
            content='Here’s what the shell reported and the current disk usage:\n\n| Item | Value 
|\n|------|-------|\n| **Shell** | `GNU bash, version 3.2.57(1)-release (arm64-apple-darwin25)` |\n| **Disk Usage**
| |\n| `/` (root) | 1.8\u202fTiB total, 11\u202fGiB used, 561\u202fGiB free (2\u202f% used) |\n| 
`/System/Volumes/Data` | 1.8\u202fTiB total, 1.2\u202fTiB used, 561\u202fGiB free (70\u202f% used) |\n| 
`/dev/disk3s6` | 1.8\u202fTiB total, 20\u202fKiB used, 561\u202fGiB free (1\u202f% used) |\n| `/dev/disk3s2` | 
1.8\u202fTiB total, 7.7\u202fGiB used, 561\u202fGiB free (2\u202f% used) |\n| `/dev/disk3s4` | 1.8\u202fTiB total, 
52\u202fMiB used, 561\u202fGiB free (1\u202f% used) |\n| `/dev/disk2s2` | 500\u202fMiB total, 6.0\u202fMiB used, 
483\u202fMiB free (2\u202f% used) |\n| `/dev/disk2s1` | 500\u202fMiB total, 5.6\u202fMiB used, 483\u202fMiB free 
(2\u202f% used) |\n| `/dev/disk2s3` | 500\u202fMiB total, 544\u202fKiB used, 483\u202fMiB free (1\u202f% used) |\n|
**Other** | `devfs`, `map auto_home`, and other s

Here’s what the shell reported and the current disk usage:

| Item | Value |
|------|-------|
| **Shell** | `GNU bash, version 3.2.57(1)-release (arm64-apple-darwin25)` |
| **Disk Usage** | |
| `/` (root) | 1.8 TiB total, 11 GiB used, 561 GiB free (2 % used) |
| `/System/Volumes/Data` | 1.8 TiB total, 1.2 TiB used, 561 GiB free (70 % used) |
| `/dev/disk3s6` | 1.8 TiB total, 20 KiB used, 561 GiB free (1 % used) |
| `/dev/disk3s2` | 1.8 TiB total, 7.7 GiB used, 561 GiB free (2 % used) |
| `/dev/disk3s4` | 1.8 TiB total, 52 MiB used, 561 GiB free (1 % used) |
| `/dev/disk2s2` | 500 MiB total, 6.0 MiB used, 483 MiB free (2 % used) |
| `/dev/disk2s1` | 500 MiB total, 5.6 MiB used, 483 MiB free (2 % used) |
| `/dev/disk2s3` | 500 MiB total, 544 KiB used, 483 MiB free (1 % used) |
| **Other** | `devfs`, `map auto_home`, and other small system volumes show negligible usage. |

- **Overall**, the main data volume (`/System/Volumes/Data`) is ~70 % full.  
- The root filesystem itself is only 2 % full.  
- All system partitions are well below their capacity.

If you need more detail on a specific partition or want to clean up space, let me know!

## Recap

In this lesson, you practiced three levels of shell middleware configuration:

- **Basic host access** for simple local automation
- **Restricted + redacted setup** for safer execution
- **Maintenance shell customization** for operational tasks

You also saw the common script pattern: configure agent → invoke with messages → inspect final response.

## Suggested Practice Tasks

1. Change Example 1 to create two files and verify both exist.
2. Add one more `RedactionRule` for email-like text.
3. Update Example 3 shutdown commands to archive logs instead of deleting them.
4. Compare behavior between `/bin/zsh` and `/bin/bash` on your machine.
